# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: OPTIONAL to execute C++ code or Rust code</h2>
            <span style="color:#f71;">As an alternative, you can run it on the website given yesterday</span>
        </td>
    </tr>
</table>


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use high end models GPT 5, Claude 4.5 Sonnet, Gemini 2.5 Pro, Grok 4, which are the slightly higher priced models. The costs are still low, but if you'd prefer to keep costs ultra low, please pick lower cost models like gpt-5-nano.
            </span>
        </td>
    </tr>
</table>


In [ ]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display


In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")





In [ ]:
# Connect to client libraries

openai = OpenAI()

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"



gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)



In [ ]:
models = ["gpt-5", "gemini-2.5-pro", "qwen2.5-coder"]

clients = {"gpt-5": openai, "gemini-2.5-pro": gemini, "qwen2.5-coder": ollama}

# Ollama (local): code-specific model. Install with: ollama pull qwen2.5-coder


## And now, on with the main task


In [ ]:
language = "Rust" # or "C++"
extension = "rs" if language == "Rust" else "cpp"

system_prompt = f"""
You are a senior software developer in multiple languages and your task if to 
detect the language with which the code was written, then produce same exact code but with
write clear and concise comments on codes and also short and concise doc strings .
you strive to be clear as possible to any one reading the code.

"""

def user_prompt_for(code):
    return f"""
    Help me comment this block of code:  {code}
"""

In [ ]:
def messages_for(code):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(code)}
    ]
 

In [ ]:
def comment_code(model, code):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(code), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    return reply

In [ ]:
python_hard = """# Be careful to support large numbers

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum


n = 10000         
initial_seed = 42 
min_val = -10     
max_val = 10      

import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:

from styles import CSS

with gr.Blocks(css=CSS, theme=gr.themes.Default(primary_hue="indigo", neutral_hue="slate")) as ui:
    gr.Markdown(
        """
        # Code Auto Commenter
        Add clear, professional comments and docstrings to Python / C++ code
        """
    )

    with gr.Row(equal_height=True, elem_classes=["main-content"]):
        with gr.Column(scale=1, min_width=500):
            gr.Markdown("**Input Code**")
            code = gr.Code(
                label="",
                value=python_hard,
                language="python",
                lines=30,
                elem_classes=["card"],
                show_label=False,
            )

        with gr.Column(scale=1, min_width=500):
            gr.Markdown("**Commented Output**")
            code_out = gr.Code(
                label="",
                lines=30,
                # placeholder="Commented code appears here…",
                interactive=False,
                elem_classes=["py-out", "card"],
                show_label=False,
            )

    with gr.Row(elem_classes=["controls"]):
        with gr.Group():
            model = gr.Dropdown(
                choices=models,
                value=models[0],
                label="Model",
                container=False,
                scale=1,
                min_width=220,
            )
            code_comment_run = gr.Button(
                "Comment Code",
                variant="primary",
                elem_classes=["run-btn", "py"],
                scale=0,
            )

    # Optional: status / info footer
    gr.Markdown(
        "*Tip: Use Python syntax highlighting works best. C++ support coming soon.*",
        elem_classes=["text-dim"]
    )

    code_comment_run.click(
        fn=comment_code,
        inputs=[model, code],
        outputs=[code_out]
    )

ui.launch(inbrowser=True)

## RESULTS!

Qwen 2.5 Coder: FAIL  
Gemini 2.5 Pro: FAIL  
DeepSeek Coder v2: FAIL  
Qwen3 Coder 30B: FAIL  
Claude Sonnet 4.5: FAIL  
GPT-5: FAIL

3rd place: GPT-oss-20B: 0.000341  
2nd place: Grok 4: 0.000317  
**1st place: OpenAI GPT-OSS 120B: 0.000304**


In [ ]:
print(f"In Ed's experimenet, the GPT-OSS 120B model outcome is {33.755209/0.000304:,.0f} times faster than the Python code.")